In [1]:
import warnings
import os
import sys

warnings.filterwarnings('ignore')
current_dir = %pwd

parent_dir = os.path.abspath(os.path.join(current_dir, '../..'))
sys.path.append(parent_dir)

In [2]:
from src.main.models.bidirLSMT import BidirectionalLSTM
from src.main.utilities.utils import clean_text, stop_words_removal, stemming, tfidf_vectorizer, get_dataset
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/francesco/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
#define model and pipeline
bidirLSTM = BidirectionalLSTM(vocab_size=100000, embedding_dim=50, max_sequence_length=75, lstm_units=64)
bidirLSTM.set_pipeline([clean_text, stop_words_removal])

#import dataset and apply pipeline
inputs, targets = get_dataset(one_hot=False)
inputs = bidirLSTM.run_pipeline(inputs, save=False)

#stratified train test split
x_train, x_test, y_train, y_test = train_test_split(inputs, targets, test_size=0.2, random_state=42, stratify=targets)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/francesco/nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/francesco/nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/francesco/nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/francesco/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/francesco/nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/francesco/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/francesco/nltk_data...
[nltk_data]   Package stopwords i

Pipeline execution time: 0:00:07.526979


In [17]:
print(inputs[0])
print(targets.head(1))

23 funniest tweets cats dogs week sept 1723 dog dont understand could eaten
2    Entertainment
Name: category, dtype: object


In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def tokenize(sequences):
    # Tokenization
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sequences)
    sequences_as_integers = tokenizer.texts_to_sequences(sequences)
    
    # Padding
    max_sequence_length = 100  # Choose an appropriate value based on your data
    return pad_sequences(sequences_as_integers, maxlen=max_sequence_length)

In [5]:
x_train = tokenize(x_train)
x_test = tokenize(x_test)

In [6]:
bidirLSTM.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, None, 50)       │     5,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, None, 128)      │        58,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,158,341 (19.68 MB)

 Trainable params: 5,158,341 (19.68 MB)

 Non-trainable params: 0 (0.00 B)

None


In [12]:
#bidirLSTM.fit(x_train, y_train)
bidirLSTM.bidirLSTM.fit(x_train, y_train, batch_size=256, epochs=3, validation_data=(x_test, y_test))

Epoch 1/3
339/339 ━━━━━━━━━━━━━━━━━━━━ 142s 419ms/step - accuracy: 0.6924 - loss: 0.8388 - val_accuracy: 0.3016 - val_loss: 1.8015
Epoch 2/3
339/339 ━━━━━━━━━━━━━━━━━━━━ 136s 401ms/step - accuracy: 0.8124 - loss: 0.5392 - val_accuracy: 0.2950 - val_loss: 2.1953
Epoch 3/3
339/339 ━━━━━━━━━━━━━━━━━━━━ 133s 392ms/step - accuracy: 0.8854 - loss: 0.3371 - val_accuracy: 0.2901 - val_loss: 2.7401


In [ ]:
#model predictions

y_pred = bidirLSTM.predict(x_test)

results = bidirLSTM.compute_performance(y_test, y_pred)
bidirLSTM.print_performance(results)


AttributeError: 'History' object has no attribute 'predict'